In [1]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 90

In [2]:
# Get our block frost api key from the file
with open('keys.txt', 'r') as file:
    api_key = file.read().replace('\n', '')
api_key = api_key.split('=')[1]

In [3]:
from blockfrost import BlockFrostApi, ApiError, ApiUrls

api = BlockFrostApi(
	project_id=api_key,
	base_url=ApiUrls.mainnet.value,
)

In [4]:
try:
    health = api.health()
    print(health)   # prints object:    HealthResponse(is_healthy=True)
    health = api.health(return_type='json') # Can be useful if python wrapper is behind api version
    print(health)   # prints json:      {"is_healthy":True}
    health = api.health(return_type='pandas')
    print(health)   # prints Dataframe:         is_healthy
                    #                       0         True

    
    account_rewards = api.account_rewards(
        stake_address='stake1ux3g2c9dx2nhhehyrezyxpkstartcqmu9hk63qgfkccw5rqttygt7',
        count=20,
        return_type='pandas'
    )
    print(account_rewards)  # prints 221
    print(len(account_rewards))  # prints 20

    account_rewards = api.account_rewards(
        stake_address='stake1ux3g2c9dx2nhhehyrezyxpkstartcqmu9hk63qgfkccw5rqttygt7',
        count=20,
        gather_pages=True, # will collect all pages
    )
    print(account_rewards[0].epoch)  # prints 221
    print(len(account_rewards))  # prints 57

    address = api.address(
        address='addr1qxqs59lphg8g6qndelq8xwqn60ag3aeyfcp33c2kdp46a09re5df3pzwwmyq946axfcejy5n4x0y99wqpgtp2gd0k09qsgy6pz')
    print(address.type)  # prints 'shelley'
    for amount in address.amount:
        print(amount.unit)  # prints 'lovelace'

except ApiError as e:
    print(e)

Namespace(is_healthy=True)
{'is_healthy': True}
   is_healthy
0        True
    epoch    amount                                            pool_id
0     221  16576013  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
1     222  17132278  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
2     223  15649019  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
3     224  17281719  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
4     225  18604433  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
5     226  12591619  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
6     227  17166845  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
7     228  17391311  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
8     231  13442322  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
9     234   4412912  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
10    235   5175375  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
11    236   8040874  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
1

# This code below is used to get all the Pool_ids (bech32 format) for all registered pools on Cardano

In [5]:
df = pd.DataFrame()

for i in range(1,100):
        try:
                pool_ids = api.pools(
                        gather_pages=True,
                        count=100,
                        page=i,)
                if len(pool_ids)==0:
                        break
                pools_list = pd.DataFrame(pool_ids)
                df = df.append(pools_list)
        except ApiError as e:
                print(e)


In [6]:
df

,0
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...
4,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...
...,...
18,pool1pnt0yy2znty3ewmqht5y7vq880mznh0y2cp6awevl...
19,pool1hjlqkwx604reu9q2azkmqp9ek7f5yy37qz662afzl...
20,pool1wj2j9zfv9mly5m7ax5fen0r8rp7tjj9uyz5gnwmyf...
21,pool178jmsy6t03udg2urvckseyync6y2jkq26haj7vxtw...


## Now let's get the list of retiring pools and then we will cross reference it with the list of all the pools and then remove any retiring pools

In [7]:
df2 = pd.DataFrame()

for i in range(1,50):
        try:
                retired_pools = api.pools_retiring(gather_pages=True, count=100, page=i)
                pools = pd.DataFrame(retired_pools)
                df2 = df2.append(pools)
        except ApiError as e:
                print(e)

In [8]:
df2

,0
0,Namespace(pool_id='pool1pxsvz3yy606zr6qnhl2snv...


In [9]:
df.iloc[0,0]

'pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087fdld7yws0xt'

In [20]:
# df3 = pd.DataFrame()
# df4 = pd.DataFrame()

# for i in range(0,len(df)):
#         try:
#                 pool_info = api.pool(pool_id=df.iloc[i,0], return_type='pandas')
#                 relays_info = api.pool_relays(pool_id=df.iloc[i,0], return_type='json')
#                 if len(relays_info) > 0:
#                         t = 0
#                         while t < len(relays_info):
#                                 df3 = df3.append(pool_info, ignore_index=True)
#                                 df4 = df4.append(pd.Series(relays_info[t]), ignore_index=True)
#                                 t += 1
#                 time.sleep(0.25)

#         except ApiError as e:
#                 if e.status_code == 402 or e.status_code == 429:
#                         break
#                 print(e)

# result = pd.concat([df3, df4 ], axis=1)

# result.to_json(r'pools_relays_info_latest.json', orient='records')


In [16]:
def get_pool_relay_data(dataframe):
        
        df = pd.DataFrame()
        df2 = pd.DataFrame()
        
        for i in range(0,len(dataframe)):
                try:
                        pool_info = api.pool(pool_id=dataframe.iloc[i,0], return_type='pandas')
                        relays_info = api.pool_relays(pool_id=dataframe.iloc[i,0], return_type='json')
                        if len(relays_info) > 0:
                                t = 0
                                while t < len(relays_info):
                                        df = df.append(pool_info, ignore_index=True)
                                        df2 = df2.append(pd.Series(relays_info[t]), ignore_index=True)
                                        t += 1
                                        time.sleep(0.25)
                except ApiError as e:
                        if e.status_code == 402 or e.status_code == 429:
                                break
                        print(e)
        result = pd.concat([df, df2 ], axis=1)
        return result.to_json(r'blockfrost_pool_relay_info_latest',orient='records')
        

In [17]:
get_pool_relay_data(df)

In [18]:
alldata = pd.read_json(r'blockfrost_pool_relay_info_latest.json', orient='records')

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement,ipv4,ipv6,dns,dns_srv,port
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,3972,32797569479332,1.378547e-03,4.903471e-01,3198,16401223192227,0.001247,100000000000,572478193547,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,[stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edj...,[841cca81da918feb9fa7257a34630eac95794be712ed3...,[],54.220.20.40,None,None,None,3002
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1731,19363531997705,8.138880e-04,2.894987e-01,168,18304506933371,0.001391,510000000000,561624278408,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,[stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389p...,[a96c79773b7506211eb56bf94886a2face17657d1009f...,[],None,None,relays.mainnet.stakenuts.com,None,3001
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,74,558085506979,2.345740e-05,8.343778e-03,15,550355739333,0.000042,50100000000,552069493858,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,[stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdx...,[37f5682e2622c821175bab9d5637b7321ecacc5ec6636...,[],None,None,relay.zenithpool.io,None,31400
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,253,1845839071889,7.758430e-05,2.759662e-02,717,1195539828288,0.000091,200000000000,205479488742,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,[stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0m...,[e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a8...,[],95.183.53.20,None,None,None,6060
4,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,253,1845839071889,7.758430e-05,2.759662e-02,717,1195539828288,0.000091,200000000000,205479488742,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,[stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0m...,[e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a8...,[],95.217.222.194,None,None,None,6060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5549,pool1hjlqkwx604reu9q2azkmqp9ek7f5yy37qz662afzl...,bcbe0b38da7d479e140ae8adb004b9b79342123e00b5a5...,9e2dd26107ad95ef05522e49879f4b2285811c0eaa504f...,0,1000000000,4.200000e-08,1.495070e-05,1,0,0.000000,1000000000,1000000000,1.000,500000000,stake1u95ggvstszjdwm2hmtpvcscjvne6fcamk2mqx2u3...,[stake1u95ggvstszjdwm2hmtpvcscjvne6fcamk2mqx2u...,[9ffe3cb33bf6e114bfb5a246488e6e5bf0f7210ae5eb4...,[],None,None,managedrewards.ddns.net,None,5001
5550,pool1wj2j9zfv9mly5m7ax5fen0r8rp7tjj9uyz5gnwmyf...,749522892c2efe4a6fdd351399bc67187cb948bc20a889...,d646e0712fd9dd18655787607970bbc643f4beca617a40...,0,616723251,2.590000e-08,9.220500e-06,2,0,0.000000,100000000,509250920,0.010,340000000,stake1u8gm2glyzm0wzzlfrnzp90y4sqc46eqta5fpyqq0...,[stake1u8gm2glyzm0wzzlfrnzp90y4sqc46eqta5fpyqq...,[10daf69798f9d1245cd5d85346d21b76e114b72dbee6d...,[],None,None,relay1.bahrtec.com,None,6000
5551,pool1wj2j9zfv9mly5m7ax5fen0r8rp7tjj9uyz5gnwmyf...,749522892c2efe4a6fdd351399bc67187cb948bc20a889...,d646e0712fd9dd18655787607970bbc643f4beca617a40...,0,616723251,2.590000e-08,9.220500e-06,2,0,0.000000,100000000,509250920,0.010,340000000,stake1u8gm2glyzm0wzzlfrnzp90y4sqc46eqta5fpyqq0...,[stake1u8gm2glyzm0wzzlfrnzp90y4sqc46eqta5fpyqq...,[10daf69798f9d1245cd5d85346d21b76e114b72dbee6d...,[],None,None,relay2.bahrtec.com,None,6001
5552,pool178jmsy6t03udg2urvckseyync6y2jkq26haj7vxtw...,f1e5b8134b7c78d42b8